In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


In [3]:
diabets_cleared = diabetes.drop_duplicates()
diabets_cleared.shape[0]

768

In [4]:
low_information_cols = [] 

#цикл по всем столбцам
for col in diabets_cleared.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabets_cleared[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabets_cleared[col].nunique() / diabets_cleared[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
        
diabets_cleared = diabets_cleared.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {diabets_cleared.shape[1]}')
display(diabets_cleared)       

Gender: 100.0% одинаковых значений
Результирующее число признаков: 9


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98,58,33,190,34.0,0.430,43,0
1,2,112,75,32,0,35.7,0.148,21,0
2,2,108,64,0,0,30.8,0.158,21,0
3,8,107,80,0,0,24.6,0.856,34,0
4,7,136,90,0,0,29.9,0.210,50,0
...,...,...,...,...,...,...,...,...,...
763,5,139,64,35,140,28.6,0.411,26,0
764,1,96,122,0,0,22.4,0.207,27,0
765,10,101,86,37,0,45.6,1.136,38,1
766,0,141,0,0,0,42.4,0.205,29,1


In [5]:
cols_null_percent = diabets_cleared.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
display(cols_with_null)

Series([], dtype: float64)

In [6]:
diabets_cleared = diabets_cleared.replace(
    {
    'Glucose' : { 0 : np.nan},
    'BloodPressure' : { 0 : np.nan},
    'SkinThickness' : { 0 : np.nan},
    'Insulin' : { 0 : np.nan}, 
    'BMI' : { 0 : np.nan}
    }
)
#display(diabets_cleared)
display(diabets_cleared['Insulin'].isnull().mean().round(2))
   

0.49

In [12]:
thresh = diabets_cleared.shape[0]*0.7
diabets_cleared = diabets_cleared.dropna(thresh=thresh, axis=1)
diabets_cleared.isnull().mean()
diabets_cleared.shape[1]

8

In [14]:
thresh2 = diabets_cleared.shape[1] - 2
diabets_cleared = diabets_cleared.dropna(thresh=thresh2, axis=0)
diabets_cleared.shape[0]
diabets_cleared.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98.0,58.0,33.0,34.0,0.430,43,0
1,2,112.0,75.0,32.0,35.7,0.148,21,0
2,2,108.0,64.0,NaN,30.8,0.158,21,0
3,8,107.0,80.0,NaN,24.6,0.856,34,0
4,7,136.0,90.0,NaN,29.9,0.210,50,0


In [19]:
values = {
    'Pregnancies': diabets_cleared['Pregnancies'].median(),
    'Glucose': diabets_cleared['Glucose'].median(),
    'BloodPressure': diabets_cleared['BloodPressure'].median(),
    'SkinThickness': diabets_cleared['SkinThickness'].median(),    
    'BMI': diabets_cleared['BMI'].median(),
    'DiabetesPedigreeFunction': diabets_cleared['DiabetesPedigreeFunction'].median(),
    'Age': diabets_cleared['Age'].median(),
    'Outcome': diabets_cleared['Outcome'].median(),
}
diabets_cleared = diabets_cleared.fillna(values)
diabets_cleared['SkinThickness'].mean().round(1)

29.1

In [20]:
def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr(diabets_cleared, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 87
Результирующее число записей: 674


In [26]:
def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_z_score(diabets_cleared, 'SkinThickness')
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу z-отклонения: 4
Результирующее число записей: 757
Число выбросов по методу z-отклонения: 11
Результирующее число записей: 750
Число выбросов по методу z-отклонения: 0
Результирующее число записей: 761


11

In [27]:
def outliers_iqr(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr(diabets_cleared, 'DiabetesPedigreeFunction')
outliers_classic = outliers.shape[0]
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')
outliers, cleaned = outliers_iqr(diabets_cleared, 'DiabetesPedigreeFunction', log_scale=True)
outliers_log = outliers.shape[0]
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')
display(outliers_classic - outliers_log)

Число выбросов по методу z-отклонения: 29
Результирующее число записей: 732
Число выбросов по методу z-отклонения: 0
Результирующее число записей: 761


29